# Filtro de spam


#### [Entrenamiento:](#Entrenamiento)
- def que separe una línea en palabras y las agregue a un conjunto dado por parámetro
- def de un generator que devuelva un set de palabras por cada correo de un mbox dado por parámetro
- def que devuelve Dict de la forma {'w' : n } con w = palabra y n = Hw para 'ham.mbox' o n = Sw para 'spam.box'

#### [Clasificación:](#Clasificación)
- def que devuelva P(y|xw) para cada palabra w de un conjunto de palabras, uno por cada correo de 'news.box'
- def que devuelva lista de 15 palabras de mejor clasificación individual de un correo nuevo
- def que devuelva P(y|x1, . . . , x15) de la lista anterior
- def que devuelva lista de 0 o 1 por cada correo de 'news.mbox' en el mismo orden

#### [Incorporación:](#Incorporación)
- def que devuelva un set de palabras de un 'new-spam.mbox' o 'new-ham.mbox' que contiene un solo mensaje
- def que combine el anterior conjunto al dict de entrenamiento aumentando en 1 cada valor
- def que escriba el nuevo correo 'new-spam.mbox' o 'new-ham.mbox' en el fichero que corresponda

#### [Ejemplo de uso:](#Ejemplo-de-uso-de-los-3-procedimientos)
- Ejemplo de uso de los 3 procedimientos anteriores

# Entrenamiento

### Utilidad

In [1]:
import re
import mailbox


# Recibe un string con el subject y body del un solo correo
# Devuelve un set con las palabras de ese string
# separadas según delimiter, un regex
def separate_in_words(payload, delimiter):
    words = set()
    for w in re.split(delimiter, payload):
        words.add(w)
    words.remove('')
    return words


# Recibe un string con la ruta de un .mbox
# Devuelve generator de un set de palabras por correo recorrido
def words_per_mail(mbox, delimiter):  
    for mail in mailbox.mbox(mbox):
        payload = mail['subject'] + ' ' + mail.get_payload()
        words = separate_in_words(payload, delimiter)
        yield words


# Recibe un set de palabras de un solo correo y un dict a modificar
# En el dict se acumulan las ocurrendias de las palabras del set
def merge_to_dict(words, dictionary):
    for w in words:
        dictionary[w] = dictionary[w] + 1 if w in dictionary else 1


# Recibe un string con la ruta de un .mbox
# Devuelve dict con key = <una palabra> y value = <nº de correos en los que aparece>,
# y el número de correos del mbox
def training_dict(mbox, delimiter):
    dictionary = {}
    count = 0
    for words in words_per_mail(mbox, delimiter):
        merge_to_dict(words, dictionary)
        count += 1
    return dictionary, count



### Procedimiento

In [2]:
# Devuelve dict de la forma {'word' : Sw }, el número de correos Spam,
# otro dict de la forma {'word' : Hw } y el número de correos Ham
def training(spambox, hambox, regex = '\W+'):
    
    spam_dict, s = training_dict(spambox, regex)    
    ham_dict, h = training_dict(hambox, regex)
    
    return spam_dict, s, ham_dict, h


# Clasificación

### Utilidad

In [83]:
# Devuelve lista de máximo n palabras con mejor clasificación individual
def best_N_words(words, spam_dict, ham_dict, n):
    
    bests = []  # De la forma: [(Pw1, w1), (Pw2, w2), (Pw3, w3), ...]
    for word in words:        
        sw = spam_dict[word] if word in spam_dict else 0  # Ocurrencias en Spam
        hw = ham_dict[word]  if word in ham_dict  else 0  # Ocurrencias en Ham
        
        # Bayes de P(y|x) ∈ [0,1] con 0.5 si es palabra neutra
        # P(y|x) más próximo a 1 o 0 es la mejor clasificación individual
        prob = 0.5 if sw == 0 and hw == 0 else sw/(sw+hw)
        
        p  = abs(prob - 0.5)  # Ahora 0.5 es la mejor clasificación individual, con p ∈ [0,0.5]
        
        if p != 0:
            bests.append(( p, word ))  # Excluimos las palabras neutras de p = 0
            
    # Ordenamos de forma decreciente por probabilidad y devolvemos las n primeras palabras
    return [word for probability, word in sorted(bests, reverse=True)][:n]


# Naive-Bayes de P(y|x1,...,xn) que suaviza usando el suavizado
# aditivo (de Laplace) con k=1
def naive_bayes7Fallos(bests, spam_dict, s, ham_dict, h, debug = False):
    
    debug = print if debug else lambda *a: None  # Igual a print(), o a función que no hace nada
    
    pi_sw   = pi_hw   = 1  # ∏Sw y ∏Hw para Naive Bayes
    Py = s/(s+h)
    Pnoy = h/(s+h)
    
    for word in bests:        
        sw = spam_dict[word] if word in spam_dict else 0  # Ocurrencias en Spam
        hw = ham_dict[word]  if word in ham_dict  else 0  # Ocurrencias en Ham
        
        #aplicamos el suavizado
        pi_sw *= Py*(sw+1)/(s+sw)
        pi_hw *= Pnoy*(hw+1)/(h+hw)
        
        debug('Sw:',sw, '- Hw:', hw, '-', word)
    
    probability = pi_sw / (pi_sw + pi_hw)  # Naive Bayes
    
    debug('S:', s, '- H:', h)
    debug('∏Sw:', pi_sw, '- ∏Hw:', pi_hw)        
    debug('P(y|x1...xn):',probability, '=> Spam' if probability > 0.9 else '=> Ham', '\n')
    
    return probability


def naive_bayes8Fallos(bests, spam_dict, s, ham_dict, h, debug = False):
    
    debug = print if debug else lambda *a: None  # Igual a print(), o a función que no hace nada
    
    pi_sw   = pi_hw   = 1  # ∏Sw y ∏Hw para Naive Bayes
    
    for word in bests:        
        sw = spam_dict[word] if word in spam_dict else 0  # Ocurrencias en Spam
        hw = ham_dict[word]  if word in ham_dict  else 0  # Ocurrencias en Ham
        
        # Suavizamos con el suavizado aditivo (de Laplace), con k=1   
        pi_sw *= (sw+1)/(s+sw)
        pi_hw *= (hw+1)/(h+hw)
        
        debug('Sw:',sw, '- Hw:', hw, '-', word)
    
    pi_sw *= s/(s+h)
    pi_hw *= h/(s+h)
    
    probability = pi_sw / (pi_sw + pi_hw)  # Naive Bayes
    
    debug('S:', s, '- H:', h)
    debug('∏Sw:', pi_sw, '- ∏Hw:', pi_hw)        
    debug('P(y|x1...xn):',probability, '=> Spam' if probability > 0.9 else '=> Ham', '\n')
    
    return probability

# Naive-Bayes de P(y|x1,...,xn) que suaviza creando entrada ficticia
# en el conjunto de entrenamiento por cada palabra que no se encuentra en este
def naive_bayes2(bests, spam_dict, s, ham_dict, h, debug = False):
    
    debug = print if debug else lambda *a: None  # Igual a print(), o a función que no hace nada
    
    s_extra = h_extra = 0  # Cuenta las veces que suavizamos
    pi_sw   = pi_hw   = 1  # ∏Sw y ∏Hw para Naive Bayes
    
    for word in bests:        
        sw = spam_dict[word] if word in spam_dict else 0  # Ocurrencias en Spam
        hw = ham_dict[word]  if word in ham_dict  else 0  # Ocurrencias en Ham
        
        # Suavizamos al cambiar 0 por 1 y contamos las veces que lo hacemos
        if sw==0:
            s_extra += 1
            sw = 1
        if hw==0:
            h_extra += 1
            hw = 1
        
        pi_sw *= sw
        pi_hw *= hw
        
        debug('Sw:',sw, '- Hw:', hw, '-', word)
    
    s += s_extra  # Agregamos tantos 1 como veces hemos suavizado
    h += h_extra
    
    probability = pi_sw / (pi_sw + pi_hw * (s/h) ** (len(bests)-1))  # Naive Bayes
    
    debug('S:', s, '- H:', h)
    debug('∏Sw:', pi_sw, '- ∏Hw:', pi_hw)        
    debug('P(y|x1...xn):',probability, '=> Spam' if probability > 0.9 else '=> Ham', '\n')
    
    return probability


### Procedimiento

In [86]:
# Función que recibe un mbox de correos nuevos
# Devuelve una lista de 0 o 1 por cada correo nuevo si es ham o spam
def clasification(newsbox, spam_dict, s, ham_dict, h, regex = '\W+', num = 15, debug = False):
    
    clas = []
    
    for words in words_per_mail(newsbox, regex):
        
        bests = best_N_words(words, spam_dict, ham_dict, num)
        
        probability  = naive_bayes8Fallos(bests, spam_dict, s, ham_dict, h, debug = debug)
        
        is_spam      = 1 if probability > 0.9 else 0
        
        clas.append(is_spam)
        
    return clas


# Incorporación

### Utilidad

In [5]:
# Escribe los correos del mbox ubicado en la ruta 'new_mbox'
# en el mbox ubicado en la ruta 'mbox'
def append_new_to_mbox(new_mbox, mbox):
    with open(mbox, 'a') as writable:
        with open(new_mbox) as readable:
            writable.write(readable.read())        


### Procedimiento

In [6]:
# Escribe los correos del mbox ubicado en la ruta 'new_mbox'
# en el mbox ubicado en la ruta 'mbox'
# Tras su uso, requiere volver a ejecutar training()
def incorporation(new_mbox, mbox):
    append_new_to_mbox(new_mbox, mbox)


# Ejemplo de uso de los 3 procedimientos

### Variables

- Rutas de archivos y parametros opcionales de training(), clasification() e incorporation()

In [7]:
hambox = './ham.mbox'
spambox = './spam.mbox'
newsbox = './news.mbox'
new_hambox  = './new-ham.mbox'
new_spambox = './new-spam.mbox'

# Criterio de separación alternativo
regex = '[.@_#/]?[\s]+|[,;:()<>*~][\s]*'

# Número de palabras alternativo para hacer Näive-Bayes
num = 15

### Uso de training()
- Parámetro regex es opcional, por defecto es '\W+', cualquier secuencia de caracteres no alfanúmericos.

In [8]:
# Entrenamiento
spam_dict, s, ham_dict, h = training(spambox, hambox)

print(s, h)
#print(spam_dict)
#print(ham_dict)

83 60


### Uso de clasification()
- Parámetro regex es opcional, por defecto es '\W+', cualquier secuencia de caracteres no alfanúmericos.
- Parámetro num es opcional, por defecto es 15.
- Parámetro debug es opcional, por defecto False. Imprime información sobre el cálculo de naive bayes.
- Requiere que training() haya sido ejecutado antes.

In [85]:
# Clasificación
res = clasification(newsbox, spam_dict, s, ham_dict, h, debug = True)

print(res[:21])  # Estos deberían ser detectados como spam
print(res[21:])  # Estos deberían ser detectados como ham

Sw: 1 - Hw: 0 - bonus
Sw: 2 - Hw: 0 - Games
Sw: 2 - Hw: 0 - 300
Sw: 21 - Hw: 7 - here
Sw: 59 - Hw: 28 - your
Sw: 2 - Hw: 4 - games
Sw: 2 - Hw: 1 - Click
Sw: 3 - Hw: 5 - free
Sw: 3 - Hw: 2 - Online
S: 83 - H: 60
∏Sw: 5.340892633026222e-14 - ∏Hw: 8.399846023216575e-16
P(y|x1...xn): 0.984516102021559 => Spam 

Sw: 3 - Hw: 0 - quick
Sw: 0 - Hw: 3 - performance
Sw: 2 - Hw: 0 - experts
Sw: 1 - Hw: 0 - erection
Sw: 1 - Hw: 0 - effects
Sw: 1 - Hw: 0 - Sex
Sw: 1 - Hw: 0 - Need
Sw: 2 - Hw: 7 - last
Sw: 10 - Hw: 3 - 0
Sw: 3 - Hw: 1 - better
Sw: 18 - Hw: 7 - only
Sw: 28 - Hw: 11 - com
Sw: 35 - Hw: 17 - http
Sw: 12 - Hw: 6 - take
Sw: 11 - Hw: 6 - than
S: 83 - H: 60
∏Sw: 9.240467070075916e-23 - ∏Hw: 3.947325958821725e-26
P(y|x1...xn): 0.9995730042174964 => Spam 

Sw: 2 - Hw: 0 - wc3
Sw: 2 - Hw: 0 - strength
Sw: 2 - Hw: 0 - solution
Sw: 6 - Hw: 0 - sexual
Sw: 3 - Hw: 0 - penis
Sw: 2 - Hw: 0 - orgasm
Sw: 2 - Hw: 0 - mogarenad
Sw: 3 - Hw: 0 - increased
Sw: 2 - Hw: 0 - greatly
Sw: 2 - Hw: 0 - greater
Sw

### Uso de incorporation()
- Escribe los nuevos correos en los archivos spambox y hambox
- Si tras su ejecución se quiere usar clasification() habría que ejecutar de nuevo training()

In [10]:
# Incorporación a spam.box
incorporation(new_spambox, spambox)

# Incorporación a ham.box
incorporation(new_hambox, hambox)